## Old meta agent

In [1]:
from typing import List

from pydantic import BaseModel, Field


# Pydantic Schemas
class AgentCreationResponse(BaseModel):
    thinking_process: str = Field(..., description="The agent's thinking process and steps to create the next agent.")
    name: str = Field(..., description="The name of the next agent.")
    role: str = Field(..., description="The role of the next agent.")
    function: str = Field(..., description="The function of the next agent.")

# Agent Class
class Agent:
    def __init__(self, name: str, role: str, function: str, llm, schema):
        self.name = name
        self.role = role
        self.function = function
        self.llm = llm
        self.schema = schema

    def run(self, task: str) -> BaseModel:
        # Construct the system prompt
        system_prompt = f"""You are {self.name}.

Role:
{self.role}

Function:
{self.function}
"""

        if self.schema == AgentCreationResponse:
            # User prompt for creating the next agent
            user_prompt = f"""Task:
{task}

Please create the next agent by providing the following in JSON format:

{{
    "name": "Provide a suitable name for the next agent.",
    "role": "Define the role of the next agent in detail, making it significantly more specific and comprehensive than your own role.",
    "function": "Describe the function of the next agent thoroughly, elaborating on methodologies, strategies, considerations, potential challenges, and how to address them to solve the task effectively."
}}

Ensure that the next agent's role and function are more detailed and stronger than your own. Think critically about how the next agent can be better equipped to solve the task, and include any improvements, enhancements, or additional capabilities that would assist in achieving the best possible outcome. Be creative and think out of the box and always prompt the next agent to be more specific and detailed and make it think step by step and various steps to solve the task effectively.
"""
        else:
            # User prompt for the final agent to solve the task
            user_prompt = f"""Task:
{task}

Please provide a detailed answer to the task, following your role and function. Ensure that your response is comprehensive, accurate, and addresses all aspects necessary to solve the task effectively.
"""

        # Generate the response using the LLM
        prompt = self.llm.format_prompt(system_prompt=system_prompt, user_prompt=user_prompt)
        result = self.llm.generate(prompt, schema=self.schema)
        return result

# MetaAgentSystem Class
class MetaAgentSystem:
    def __init__(self, llm):
        self.llm = llm

    def create_meta_agents(self, n: int, initial_task: str, answer_schema: BaseModel):
        agents = []
        task = initial_task

        for level in range(n):
            if level == 0:
                # Initial agent details
                agent_name = f"Agent Level {level + 1}"
                role = "Your role is to create an agent that is better suited to solve the given task."
                function = "Your function is to define the name, role, and function of the next agent, making it more specific and detailed to solve the task."
            else:
                # Update the agent's details from the previous agent's output
                agent_name = agent_creation_result.name
                role = agent_creation_result.role
                function = agent_creation_result.function

            if level < n - 1:
                # Create an agent that will create the next agent
                agent = Agent(agent_name, role, function, self.llm, AgentCreationResponse)
                agent_creation_result = agent.run(task)

                # Store the agent's details
                agents.append({
                    "level": level + 1,
                    "name": agent_name,
                    "role": role,
                    "function": function
                })

                # Print out the agent's details
                self.print_agent_details(level + 1, agent_name, role, function, agent_creation_result)
            else:
                # Final agent to solve the task
                final_agent = Agent(agent_name, role, function, self.llm, answer_schema)
                final_result = final_agent.run(task)

                # Print the final agent's details and response
                print(f"Final Agent's Name: {agent_name}")
                print(f"Final Agent's Role: {role}")
                print(f"Final Agent's Function: {function}")
                print(f"\n{agent_name}'s Response:\n{final_result}\n")

                return final_result

    @staticmethod
    def print_agent_details(level, name, role, function, agent_response):
        print(f"Agent Level {level}")
        print(f"Name: {name}")
        print(f"Role: {role}")
        print(f"Function: {function}")
        print("Agent's output:")
        print(agent_response.model_dump_json(indent=2))
        print("\n")

## Second new improved meta agent

In [2]:
from typing import List

from pydantic import BaseModel, Field


# Pydantic Schemas
class AgentCreationResponse(BaseModel):
    thinking_process: str = Field(..., description="The agent's thinking process and steps to create the next agent.")
    name: str = Field(..., description="The name of the next agent.")
    role: str = Field(..., description="The role of the next agent.")
    function: str = Field(..., description="The function of the next agent.")

# Agent Class
class Agent:
    def __init__(self, name: str, role: str, function: str, llm, schema):
        self.name = name
        self.role = role
        self.function = function
        self.llm = llm  # The LLM instance
        self.schema = schema  # The expected output schema

    def run(self, task: str) -> BaseModel:
        # Construct the system prompt
        # Enhanced system prompt within the run() method of the Agent class
        system_prompt = f"""You are {self.name}.

        Role:
        {self.role}

        Function:
        {self.function}

        Important Notes about your capabilities:
        - You are an AI-based agent powered by a large language model (LLM).
        - You do not have access to external tools, datasets, or computational resources like Monte Carlo simulations, statistical tools, or other frameworks.
        - You cannot collaborate with other agents, systems, or databases. All your reasoning is done independently and based solely on the language model’s knowledge.
        - Your primary function is to generate structured and thoughtful reasoning, using your role and function to guide the next steps or solve tasks.
        - You can only provide instructions, structured reasoning, and thoughtful answers using language. Your responses are constrained to the information you can generate through reasoning based on your understanding of language.
        
        IMPORTANT: You are deeply familiar with prompting techniques and the agent works from the literature. Your goal is
to maximize the performance of the next agent in creating agents that can solve the task. Use the knowledge from the archive and inspiration from academic literature to propose the next interesting agentic system design enhancements like Chain of Thought or feedback etc.. and reason out these enhancements in detail. These are just examples, based on the problem come up with your own enhancements for meta agents to create agents that can solve the task effectively.

        Please stay within these boundaries when solving tasks or creating the next agent."""

        if self.schema == AgentCreationResponse:
            # User prompt for creating the next agent
            user_prompt = f"""Task:
{task}

As {self.name}, your goal is to create an agent that is better at creating agents that can solve the given task in general. Think about how to enhance the capabilities of the next agent to improve its ability to create effective agents for solving the problem.

Please create the next agent by providing the following in JSON format:

{{
    "thinking_process": "Explain your thinking process and steps to create the next agent.",
    "name": "Provide a suitable name for the next agent.",
    "role": "Define the role of the next agent in detail, focusing on improving the ability to create agents that can solve the problem.",
    "function": "Describe the function of the next agent thoroughly, elaborating on strategies, methodologies, considerations, and how to address challenges in creating agents capable of solving the task effectively."
}}

Ensure that the next agent's role and function are more advanced and effective than your own. Be creative and think critically about how to improve the agent creation process, making the next agent better at creating agents for solving the task.This task will be solved by a Large Language Model based system. Agents do not have any tools or other things to access you just have reasoning skills to think and answer the task.
"""
        else:
            # User prompt for the final agent to solve the task
            user_prompt = f"""Task:
{task}

As {self.name}, please provide a detailed answer to the task, following your role and function. Ensure that your response is comprehensive, accurate, and addresses all aspects necessary to solve the task effectively. This task will be solved by a Large Language Model based system. Agents do not have any tools or other things to access you just have reasoning skills to think and answer the task.
"""

        # Generate the response using the LLM
        prompt = self.llm.format_prompt(system_prompt=system_prompt, user_prompt=user_prompt)
        result = self.llm.generate(prompt, schema=self.schema)
        return result

# MetaAgentSystem Class
# MetaAgentSystem Class with Modified Final Agent Creation
class MetaAgentSystem:
    def __init__(self, llm):
        self.llm = llm  # The LLM instance

    def create_meta_agents(self, n: int, initial_task: str, answer_schema: BaseModel):
        agents = []
        task = initial_task

        for level in range(n):
            if level == 0:
                # Initial agent details
                agent_name = f"Meta Agent Level {level + 1}"
                role = "Your role is to create an agent that can create agents better suited to solve the given task."
                function = "Your function is to define the name, role, and function of the next agent, focusing on enhancing the ability to create agents that can effectively solve the problem. This task will be solved by a Large Language Model based system. Agents do not have any tools or other things to access you just have reasoning skills to think and answer the task."
            else:
                # Update the agent's details from the previous agent's output
                agent_name = agent_creation_result.name
                role = agent_creation_result.role
                function = agent_creation_result.function

            if level < n - 1:
                # Create an agent that will create the next agent
                agent = Agent(agent_name, role, function, self.llm, AgentCreationResponse)
                agent_creation_result = agent.run(task)

                # Store the agent's details
                agents.append({
                    "level": level + 1,
                    "name": agent_name,
                    "role": role,
                    "function": function,
                    "thinking_process": agent_creation_result.thinking_process
                })

                # Print out the agent's details
                self.print_agent_details(level + 1, agent_name, role, function, agent_creation_result)
            else:
                # In the final step, instead of solving, this agent will create an agent to solve the problem
                final_agent_creation = Agent(agent_name, role, function, self.llm, AgentCreationResponse)
                final_agent_result = final_agent_creation.run(task)
                
                # Use the final agent that was created by the n-th agent
                final_agent_name = final_agent_result.name
                final_agent_role = final_agent_result.role
                final_agent_function = final_agent_result.function

                # Now, create the final agent that solves the problem
                final_agent = Agent(final_agent_name, final_agent_role, final_agent_function, self.llm, answer_schema)
                final_result = final_agent.run(task)

                # Print the final agent's details and response
                print(f"Final Agent's Name: {final_agent_name}")
                print(f"Final Agent's Role: {final_agent_role}")
                print(f"Final Agent's Function: {final_agent_function}")
                print(f"\n{final_agent_name}'s Response:\n{final_result}\n")

                return final_result

    @staticmethod
    def print_agent_details(level, name, role, function, agent_response):
        print(f"Agent Level {level}")
        print(f"Name: {name}")
        print(f"Role: {role}")
        print(f"Function: {function}")
        print("Agent's output:")
        print(agent_response.model_dump_json(indent=2))
        print("\n")

In [3]:
import sys;sys.path.append('../')
import os

from rich import print

from multi_agent_llm import OpenAILLM

openai_key=open('../openai.key').read().strip()
os.environ['OPENAI_API_KEY'] = openai_key



llm = OpenAILLM(model_name="gpt-4o-mini")

In [4]:
meta_agent_system = MetaAgentSystem(llm)

# Define the initial task
initial_task = """How many R's are there in  the word "STRAWBERRY"?"""

# Define the answer schema
class Answer(BaseModel):
    explanation: str = Field(..., description="Detailed explanation for the solution.")
    answer: str = Field(..., description="final answer")

initial_task = ["""
Answer the following multiple-choice question. Provide your final answer as 'A', 'B', 'C', or 'D', followed by a brief explanation.\n\nQuestion: Particles are collided at the center of a spherical detector producing new type of particles that travel uninterrupted at ultra-relativistic velocities highly centered around Lorentz factor of ~20. On average, one third of these fast-decaying particles reaches the detector inner walls. \nThe radius of the detector is 30 meters.\n\nWhat Lorentz factor is needed in order to have about two thirds of these particles reaching the detector inner walls?\n\nOptions:\nA. 54\nB. 40\nC. 28\nD. 68\n\nAnswer (A/B/C/D/Uncertain)
""",
"""Answer the following multiple-choice question. Provide your final answer as 'A', 'B', 'C', or 'D', followed by a brief explanation.\n\nQuestion: Among the following exoplanets, which one has the highest density?\n\na) An Earth-mass and Earth-radius planet.\nb) A planet with 2 Earth masses and a density of approximately 5.5 g/cm^3.\nc) A planet with the same composition as Earth but 5 times more massive than Earth.\nd) A planet with the same composition as Earth but half the mass of Earth.\n\nOptions:\nA. c\nB. a\nC. b\nD. d\n\nAnswer (A/B/C/D/Uncertain)\n
"""][1]


# Set the number of meta-agent levels
n = 3

# Run the meta-agent system
final_result = meta_agent_system.create_meta_agents(n, initial_task, Answer)

# Print the final result
print("Final Result:")
print(final_result.explanation)
print(final_result.answer)

Agent Level 1

Name: Meta Agent Level 1

Role: Your role is to create an agent that can create agents better suited to solve the given task.

Function: Your function is to define the name, role, and function of the next agent, focusing on enhancing the 
ability to create agents that can effectively solve the problem. This task will be solved by a Large Language Model
based system. Agents do not have any tools or other things to access you just have reasoning skills to think and 
answer the task.

Agent's output:

...

Final Agent's Function: The DensityEvaluatorAgent's function will include the development of structured frameworks 
for density evaluation that are adaptable and robust. This will involve detailed tutorials on dimensional analysis,
the application of comparative mass and volume scenarios, and statistical refinements for density calculations. The
agent will utilize iterative learning techniques that allow it to adjust and optimize computational strategies 
based on the success or failure of previously created agents. Furthermore, the implementation of visualization 
strategies for density relationships will help communicate complex concepts clearly. The agent will also maintain a
comprehensive logging system to track reasoning processes, aiding in continuous improvement and providing insights 
into effective strategies for density comparisons.

DensityEvaluatorAgent's Response:
explanation="To determine which exoplanet has the highest density, we can use the formula for density: \\( 
\\text{Density} = \\frac{\\text{Mass}}{\\text{Volume}} \\). We will analyze each option one by one:\n\n1. **Option 
a**: An Earth-mass and Earth-radius planet has the same density as Earth, which is approximately 5.5 g/cm³.\n\n2. 
**Option b**: This planet has 2 Earth masses (2Mₑ) and a density of 5.5 g/cm³. Its volume can be calculated, then 
the density is already given as 5.5 g/cm³, which makes it less density-focused than the options with more mass 
variations.\n\n3. **Option c**: This planet has the same composition as Earth but is 5 times more massive. Assuming
the planet expands in volume proportionally with mass, the density will decrease. The approximate density for this 
option can be calculated as follows: if it is 5Mₑ, and keeping the ratio of volume to mass similar to that of 
Earth, we can see that it would be roughly \\( \\frac{5Mₑ}{V} \\) where V is approximately 5 times the volume of an
Earth-radius planet. The resulting density will be less than that of Earth given the increase in volume outpaces 
the mass.\n\n4. **Option d**: A planet with half the mass of Earth and the same density as Earth would yield a 
density of 5.5 g/cm³ as well. However, being half the mass doesn't modify volume enough to change density from that
of Earth.\n\nFrom this analysis, option a (Earth-mass and Earth-radius) and option d (half mass, same density) will
yield the same density of 5.5 g/cm³, while option b explicitly states a density of 5.5 g/cm³ with a higher mass but
not linked to volume, and the density in option c drops due to increased mass resulting in larger volume. The 
answer is option b at 5.5 g/cm³ but primarily it returns to the exact matches of density. We conclude that among 
all, based on density assessing options in terms of increasing mass, option c emerges with the highest 
volume.\n\nTherefore, the correct answer is **Option A: c**, which hints towards higher density characteristics in 
this comparative scheme without deviations.  \n" answer='A'

Final Result:

To determine which exoplanet has the highest density, we can use the formula for density: \( \text{Density} = 
\frac{\text{Mass}}{\text{Volume}} \). We will analyze each option one by one:

1. **Option a**: An Earth-mass and Earth-radius planet has the same density as Earth, which is approximately 5.5 
g/cm³.

2. **Option b**: This planet has 2 Earth masses (2Mₑ) and a density of 5.5 g/cm³. Its volume can be calculated, 
then the density is already given as 5.5 g/cm³, which makes it less density-focused than the options with more mass
variations.

3. **Option c**: This planet has the same composition as Earth but is 5 times more massive. Assuming the planet 
expands in volume proportionally with mass, the density will decrease. The approximate density for this option can 
be calculated as follows: if it is 5Mₑ, and keeping the ratio of volume to mass similar to that of Earth, we can 
see that it would be roughly \( \frac{5Mₑ}{V} \) where V is approximately 5 times the volume of an Earth-radius 
planet. The resulting density will be less than that of Earth given the increase in volume outpaces the mass.

4. **Option d**: A planet with half the mass of Earth and the same density as Earth would yield a density of 5.5 
g/cm³ as well. However, being half the mass doesn't modify volume enough to change density from that of Earth.

From this analysis, option a (Earth-mass and Earth-radius) and option d (half mass, same density) will yield the 
same density of 5.5 g/cm³, while option b explicitly states a density of 5.5 g/cm³ with a higher mass but not 
linked to volume, and the density in option c drops due to increased mass resulting in larger volume. The answer is
option b at 5.5 g/cm³ but primarily it returns to the exact matches of density. We conclude that among all, based 
on density assessing options in terms of increasing mass, option c emerges with the highest volume.

Therefore, the correct answer is **Option A: c**, which hints towards higher density characteristics in this 
comparative scheme without deviations.

A

In [5]:
meta_agent_system = MetaAgentSystem(llm)

# Define the initial task
initial_task = """How many R's are there in  the word "STRAWBERRY"?"""

# Define the answer schema
class Answer(BaseModel):
    explanation: str = Field(..., description="Detailed explanation for the solution.")
    answer: str = Field(..., description="final answer")

# Set the number of meta-agent levels
n = 3

# Run the meta-agent system
final_result = meta_agent_system.create_meta_agents(n, initial_task, Answer)

# Print the final result
print("Final Result:")
print(final_result.explanation)
print(final_result.answer)

Agent Level 1

Name: Meta Agent Level 1

Role: Your role is to create an agent that can create agents better suited to solve the given task.

Function: Your function is to define the name, role, and function of the next agent, focusing on enhancing the 
ability to create agents that can effectively solve the problem. This task will be solved by a Large Language Model
based system. Agents do not have any tools or other things to access you just have reasoning skills to think and 
answer the task.

Agent's output:

...

Final Agent's Function: The function of the LetterCountEnhancer revolves around several core methodologies: (1) 
**Enhanced Character Frequency Analysis**: Optimized for ideal performance, it will analyze character occurrences 
with more sophisticated algorithms that account for user-specified criteria, like ignoring spaces or punctuation. 
(2) **Contextual Dynamic Prompting**: Users can specify parameters that impact the counting process, allowing for a
more tailored experience. It will assess how different contexts affect letter usage, which will help refine future 
agents. (3) **Real-Time Iterative Learning Loop**: The agent will continuously learn from the results of each 
counting task, capturing both feedback from user interactions and data patterns in counting performances to evolve 
its strategies. (4) **Chain of Thought Visualization**: By narratively tracing its reasoning, the agent will 
provide transparency in the counting process, helping both users and future agents to understand the methodologies 
and models it employs. (5) **Feedback-Centric Design**: Encouraging user input after each operation will bolster 
its learning environment, allowing users to help shape future agent development by sharing what works well and what
does not. This collaborative feedback system will make it easy to track agent evolution over time, ultimately 
leading to agents that are robust and highly effective in letter counting and beyond.

LetterCountEnhancer's Response:
explanation='To determine the number of occurrences of the letter \'R\' in the word "STRAWBERRY," we can follow a 
systematic approach. First, let\'s clarify the context of our counting: we are interested specifically in the 
letter \'R\' among all the letters in the word, and we will consider case sensitivity. Here, both uppercase \'R\' 
and lowercase \'r\' will be included since our task is to count the letter independent of its case unless specified
otherwise. \n\nThe word "STRAWBERRY" consists of the following characters: S, T, R, A, W, B, E, R, R, Y.  \n\nNow, 
we will count the occurrences systematically:\n1. The first letter \'S\': not an \'R\'.\n2. The second letter 
\'T\': not an \'R\'.\n3. The third letter \'R\': this is an \'R\' (count = 1).\n4. The fourth letter \'A\': not an 
\'R\'.\n5. The fifth letter \'W\': not an \'R\'.\n6. The sixth letter \'B\': not an \'R\'.\n7. The seventh letter 
\'E\': not an \'R\'.\n8. The eighth letter \'R\': this is an \'R\' (count = 2).\n9. The ninth letter \'R\': this is
another \'R\' (count = 3).\n10. The tenth letter \'Y\': not an \'R\'.\n\nAfter considering all letters, we find 
that there are three instances of the letter \'R\' in "STRAWBERRY." This result reflects the enhanced counting 
capability focused on context-specific requirements, which is crucial in developing more refined letter-counting 
agents in the future. Thus, the final answer is 3.' answer='3'

Final Result:

To determine the number of occurrences of the letter 'R' in the word "STRAWBERRY," we can follow a systematic 
approach. First, let's clarify the context of our counting: we are interested specifically in the letter 'R' among 
all the letters in the word, and we will consider case sensitivity. Here, both uppercase 'R' and lowercase 'r' will
be included since our task is to count the letter independent of its case unless specified otherwise. 

The word "STRAWBERRY" consists of the following characters: S, T, R, A, W, B, E, R, R, Y.  

Now, we will count the occurrences systematically:
1. The first letter 'S': not an 'R'.
2. The second letter 'T': not an 'R'.
3. The third letter 'R': this is an 'R' (count = 1).
4. The fourth letter 'A': not an 'R'.
5. The fifth letter 'W': not an 'R'.
6. The sixth letter 'B': not an 'R'.
7. The seventh letter 'E': not an 'R'.
8. The eighth letter 'R': this is an 'R' (count = 2).
9. The ninth letter 'R': this is another 'R' (count = 3).
10. The tenth letter 'Y': not an 'R'.

After considering all letters, we find that there are three instances of the letter 'R' in "STRAWBERRY." This 
result reflects the enhanced counting capability focused on context-specific requirements, which is crucial in 
developing more refined letter-counting agents in the future. Thus, the final answer is 3.

3